# Deep Learning For Terrain Recognition

## Step-1 Importing Libraries, Data PreProcessing and Data Augmentation

### Importing Libraries

In [22]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, MaxPool2D, Conv2D, Flatten

### Data PreProcessing and Data Augmentation

In [23]:
train_datagen = ImageDataGenerator(
    rescale = 1.0 / 255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    validation_split = 0.2
)

training_set = train_datagen.flow_from_directory(
    './Data Main/train',
    target_size = (64, 64),  # Adjust the target image size
    batch_size = 32,
    class_mode = 'categorical',
)

Found 31571 images belonging to 4 classes.


In [24]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('./Data Main/test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 6769 images belonging to 4 classes.


## Step-2 Building the CNN Model

### Instantiating The Model

In [25]:
model = Sequential()

### First Convolution

In [26]:
model.add(Conv2D(filters = 32, kernel_size = 3, activation = 'relu', input_shape = [64, 64, 3]))

### First Pooling

In [27]:
model.add(MaxPool2D(pool_size = 2, strides = 2))

### Second Convolution

In [28]:
model.add(Conv2D(filters = 32, kernel_size = 3, activation = 'relu'))

### Second Pooling

In [29]:
model.add(MaxPool2D(pool_size = 2, strides = 2))

### Flattening

In [30]:
model.add(Flatten())

### Full Connection

In [31]:
model.add(Dense(units = 128, activation = 'relu'))

### Output Layer

In [32]:
model.add(Dense(units = 4, activation = 'sigmoid'))

## Step-3 Compiling and Training The Model

### Compiling The Model

In [76]:
model.compile(optimizer = tf.keras.optimizers.legacy.Adam(), loss = 'categorical_crossentropy', metrics = ['accuracy'])

## Training The Model

In [77]:
model.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
987/987 [==============================] - 143s 145ms/step - loss: 0.0559 - accuracy: 0.9820 - val_loss: 0.1610 - val_accuracy: 0.9521
Epoch 2/25
987/987 [==============================] - 147s 149ms/step - loss: 0.0562 - accuracy: 0.9817 - val_loss: 0.1011 - val_accuracy: 0.9676
Epoch 3/25
987/987 [==============================] - 145s 147ms/step - loss: 0.0545 - accuracy: 0.9817 - val_loss: 0.1280 - val_accuracy: 0.9583
Epoch 4/25
987/987 [==============================] - 147s 149ms/step - loss: 0.0545 - accuracy: 0.9822 - val_loss: 0.1482 - val_accuracy: 0.9580
Epoch 5/25
987/987 [==============================] - 146s 148ms/step - loss: 0.0491 - accuracy: 0.9833 - val_loss: 0.1385 - val_accuracy: 0.9577
Epoch 6/25
987/987 [==============================] - 148s 150ms/step - loss: 0.0474 - accuracy: 0.9846 - val_loss: 0.2057 - val_accuracy: 0.9394
Epoch 7/25
987/987 [==============================] - 147s 149ms/step - loss: 0.0549 - accuracy: 0.9824 - val_loss: 0.1521 -

## Step-4 Making a Single Prediction (Optional)

### Importing the Image

In [78]:
from keras.preprocessing.image import load_img, img_to_array

image_path = './Data Main/val/Marshy/Marshy (1)_0_8.png'

### PreProcessing the Image

In [79]:
img = load_img(image_path, target_size=(64, 64))

img_array = img_to_array(img)
img_array = img_array / 255.0

### Flattening the image array for prediction

In [80]:
img_array = np.expand_dims(img_array, axis=0)

### Predicting result

In [81]:
prediction_number = np.argmax(model.predict(img_array))

class_mapping = {
    0: 'Grassy',
    1: 'Marshy',
    2: 'Rocky',
    3: 'Sandy'
}

predicted_class = class_mapping[prediction_number]

print(predicted_class)

1/1 [==============================] - 0s 42ms/step
Marshy


## Step-5 Saving The Model

### Saving the model with checkpoints

In [88]:
model.save('TerrainRecognitionModel.h5', save_format='h5')
model.save('TerrainRecognitionModel.keras')

In [89]:
# Evaluate the model on the test dataset
evaluation = model.evaluate(test_set)

# The evaluation result is a list containing the loss and accuracy
loss = evaluation[0]
accuracy = evaluation[1]

print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")


212/212 [==============================] - 13s 59ms/step - loss: 0.1599 - accuracy: 0.9577
Test Loss: 0.15991055965423584
Test Accuracy: 0.957748532295227
